In [5]:
import datetime
import time
import os
# third party libraries
import cv2
import numpy as np
from ultralytics import YOLO

class Config:
    DEBUG_MODE = True
    NUM_ROUND = 4
    DEFAULT_COLOR = cv2.COLOR_BGR2RGB
    STITCH_THESHOLD = 0.5
    DETECT_CONF = 0.5
    RESULTS_DIR = "results/" # append / at the end

In [6]:
"""Helper functions"""

def depug_print(str):
    if Config.DEBUG_MODE:
        now = datetime.datetime.now().strftime("%H:%M:%S")
        print(f"{now} - {str}")
    
def normal_print(str):
    print(f"{str}")

def load_convert(path):
    """
    Loads and converts an image to RGB
    """
    return cv2.cvtColor(cv2.imread(path), Config.DEFAULT_COLOR)

def ensure_results_directory():
    """Create the results directory if it doesn't exist."""
    os.makedirs(Config.RESULTS_DIR, exist_ok=True)

ensure_results_directory()

In [7]:
#############
# Stitching #
def stitch_match_2(left, right):
    """Function to stitch two images together"""
    start = time.time()

    stitcher = cv2.Stitcher_create(cv2.Stitcher_PANORAMA)
    stitcher.setPanoConfidenceThresh(Config.STITCH_THESHOLD)
    status, res_I = stitcher.stitch([left, right])
    end = time.time()

    if status == cv2.Stitcher_OK:
        depug_print(f"stitch_match_2 took: {round(end-start, Config.NUM_ROUND)} seconds")
        return res_I
    else:
        if status == cv2.Stitcher_ERR_NEED_MORE_IMGS:
            depug_print("Error: Need more images to stitch.")
        elif status == cv2.Stitcher_ERR_HOMOGRAPHY_EST_FAIL:
            depug_print("Error: Homography estimation failed.")
        elif status == cv2.Stitcher_ERR_CAMERA_PARAMS_ADJUST_FAIL:
            depug_print("Error: Camera parameters adjustment failed.")
        else:
            depug_print(f"Error during stitching: {status}")
        return None

def stitch_3_imgs(images_folder):
    """
    Requires 3 ordered RGB images
    Returns stitched image if successful, otherwise None
    """
    left = load_convert(f'{images_folder}/1.png')
    center = load_convert(f'{images_folder}/2.png')
    right = load_convert(f'{images_folder}/3.png')
    
    depug_print("Stitching left and center...")
    left_res = stitch_match_2(left, center)

    if left_res is None:
        depug_print("Stitching failed between left and center!")
        return

    depug_print("Stitching center and right...")
    right_res = stitch_match_2(center, right)

    if right_res is None:
        depug_print("Stitching failed between center and right!")
        return

    depug_print("Stitching final result between left-center and right...")
    final_stitch = stitch_match_2(left_res, right_res)

    if final_stitch is None:
        depug_print("Final stitching failed!")
    else:
        depug_print("Stitching completed successfully!")
        cv2.imwrite(f"{Config.RESULTS_DIR}/stitched.jpg", final_stitch)

    return final_stitch
#############

##################
# Edge Detection #
def apply_gaussian_blur(image, sigma):
    """Apply Gaussian Blur to the image with a given sigma."""
    return cv2.GaussianBlur(image, (0, 0), sigmaX=sigma, sigmaY=sigma)

def difference_of_gaussians(image, sigma1, sigma2):
    """Calculate the Difference of Gaussians (DoG) for the image."""
    blur1 = apply_gaussian_blur(image, sigma1)
    blur2 = apply_gaussian_blur(image, sigma2)
    return blur1 - blur2

def clean_dog_image(dog_image):
    """Apply morphological closing to the DoG image."""
    kernel = np.ones((3, 3), np.uint8)
    return cv2.morphologyEx(dog_image, cv2.MORPH_CLOSE, kernel)

def edge_detection_pipeline(image, sigma1, sigma2):
    """Complete edge detection pipeline using DoG."""
    
    dog_image = difference_of_gaussians(image, sigma1, sigma2)
    cleaned_dog = clean_dog_image(dog_image)

    result_filename = f'{Config.RESULTS_DIR}edge_result.jpg'
    cv2.imwrite(result_filename, cleaned_dog)
    normal_print(f"Result saved to {result_filename}")
    return cleaned_dog
################

#################
# Predict people#
def predict_people(image, model='yolov8x'):
    """Load the YOLO model, predict, and save the result image with bounding boxes."""
    model = YOLO(f'models/{model}.pt')
    results = model.predict(source=image, conf=Config.DETECT_CONF)

    for i, result in enumerate(results):
        result.save(filename=f'{Config.RESULTS_DIR}detect_result.jpg')

    normal_print(f"Images saved to {Config.RESULTS_DIR}detect_result.jpg")
    return results


In [8]:
# Main program
#* Note: all images saved locally in `/results` folder

# must include in order, left to right 1, 2, 3
stitch_folder = 'test'
stitched = stitch_3_imgs(images_folder=stitch_folder)

edge = edge_detection_pipeline(image=stitched, sigma1=1.0, sigma2=2.0)

# uses by default the largest model
predict = predict_people(image=stitched, model='yolov8x')

22:56:45 - Stitching left and center...
22:56:45 - stitch_match_2 took: 0.5233 seconds
22:56:45 - Stitching center and right...
22:56:46 - stitch_match_2 took: 0.4601 seconds
22:56:46 - Stitching final result between left-center and right...
22:56:47 - stitch_match_2 took: 0.8489 seconds
22:56:47 - Stitching completed successfully!
Result saved to results/edge_result.jpg

0: 224x640 4 persons, 479.0ms
Speed: 2.4ms preprocess, 479.0ms inference, 0.8ms postprocess per image at shape (1, 3, 224, 640)
Images saved to results/detect_result.jpg
